In [ ]:
import mdtraj as md
import mdciao

### Download example data and load into the namespace

In [ ]:
mdciao.examples.fetch_example_data()
traj = md.load("mdciao_example/traj.xtc",top="mdciao_example/prot.pdb")

### Create consensus labeler objects

In [ ]:
GPCR = mdciao.nomenclature.LabelerGPCR("adrb2_human")
CGN = mdciao.nomenclature.LabelerCGN("3SN6")

### Guess molecular fragments
This would be done anyway by the `mdciao.cli.interface` call in the cell below, here we do it have the fragments defined in the namespace

In [ ]:
fragments = mdciao.fragments.get_fragments(traj.top);
fragment_names = ["Galpha", "Gbeta", "Ggamma","B2AR","P0G"]

### Compute G$\alpha$-B2AR interface
Using the above fragment definitions

In [ ]:
intf = mdciao.cli.interface(traj,
                            title="3SN6 beta2AR-Galpha interface",
                            fragments=fragments, fragment_names = fragment_names,                             
                            frag_idxs_group_1=[0], 
                            frag_idxs_group_2=[3],
                            GPCR_uniprot=GPCR, CGN_PDB=CGN,
                            accept_guess=True, no_disk=True, figures=False)

### Plot each residues's participation in the interface

In [ ]:
ifig = intf.plot_frequency_sums_as_bars(3.5, title_str = intf.name, 
                                 list_by_interface=True, 
                                 interface_vline=True);
ifig.figure.savefig("intf.svg")

### Plot contact matrix

In [ ]:
ifig = intf.plot_interface_frequency_matrix(3.5, grid=True, pixelsize=.5);
ifig[0].savefig("matrix.svg")

### Flareplot
We combine a lot of information into one single flareplot:  
* the molecular topology with sub-fragments and consensus labels, 
* the secondary structure, 
* the individual contact-pairs 
* the participation of each residue in the interface.

In [ ]:
ifig, iax = intf.plot_freqs_as_flareplot(3.5, 
                                         fragments=fragments, fragment_names = fragment_names,  
                                         scheme="consensus", consensus_maps=[GPCR, CGN], 
                                         aura=intf.frequency_sum_per_residue_idx_dict(3.5,return_array=True),
                                         SS=True)

### Coarse-Grained Frequencies and Flareplots

In [ ]:
intf.frequency_as_contact_matrix_CG(3.5, fragments=fragments, fragment_names = fragment_names,
                                    consensus_labelers=[GPCR, CGN],
                                    interface=True).round(1).replace(0,"")  

ifig, iax = intf.plot_freqs_as_flareplot(3.5, 
                                         fragments=fragments, fragment_names = fragment_names,  
                                         consensus_maps=[GPCR, CGN],                                         
                                         coarse_grain=True,
                                        )


### Grab a representative frame
This frame will be used to plot the interface frequencies as a 3D heatmap (see `frequency_to_bfactor` below).

In [ ]:
repframe = intf.repframes(return_traj=True)[-1][0]

### Save the interface as a heatmap and view externally

In [ ]:
intf.frequency_to_bfactor(3.5, pdbfile="interface_heatmap.pdb", 
                          geom=repframe, 
                          interface_sign=True
                         );

### Save all mdciao objects for later reuse
We can save all mdciao objects to numpy .npy (pickle) files and later reload them without having to compute everything again.

In [ ]:
import numpy as np
np.save("GPCR.npy", GPCR)
np.save("CGN.npy",CGN)
np.save("intf.npy",intf)

### Compare to a different setup using violin plots

In [ ]:
mdciao.examples.fetch_example_data(url='http://proteinformatics.org/mdciao/mdciao_example2.zip')
traj2 = md.load("mdciao_example2/traj2.xtc",top="mdciao_example2/prot2.pdb")

In [ ]:
intf_site = {"name":"intf", 
             "pairs": {"residx" : intf.res_idxs_pairs}}

In [ ]:
site1 = mdciao.cli.sites([intf_site], traj2, 
                  no_disk=True, figures=False, 
                  #fragments=fragments, fragment_names = fragment_names,                             
                  GPCR_uniprot=GPCR, CGN_PDB=CGN,
                  accept_guess=True)["intf"]

In [ ]:
res = mdciao.plots.compare_violins({
                            "dataset 1": intf, 
                            "dataset 2": site1}, 
                            defrag=None,        #keep fragment names
                            key_separator=None, #keep residue label order
                            sort_by="residue",
                            ctc_cutoff_Ang=4, 
                            remove_identities=True,
                            identity_cutoff=.90,
                            ymax=20, 
                            colors="Set2"
                            );